In [ ]:
# Animated histogram of positions in a 1D random walk
# - 1000 particles
# - Start at 0
# - Probability p_right to move +1 and (1 - p_right) to move -1
# - Shows how the distribution evolves over time (single-axes animation)

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display

# ---- Parameters (feel free to tweak) ----
n_particles = 1000
n_steps = 120
p_right = 0.5  # change this to bias the walk (e.g., 0.6)
p_left = 1.0 - p_right

# ---- Simulate all walks at once ----
# steps: shape (n_particles, n_steps) with values -1 or +1
steps = np.random.choice([-1, 1], size=(n_particles, n_steps), p=[p_left, p_right])
# cumulative positions over time; include t=0 column of zeros
positions_over_time = np.concatenate(
    [np.zeros((n_particles, 1), dtype=int), np.cumsum(steps, axis=1)],
    axis=1
)  # shape: (n_particles, n_steps+1)

# Precompute sensible histogram bins spanning possible positions
pos_min = positions_over_time.min()
pos_max = positions_over_time.max()
# Use integer bin edges covering the span
bins = np.arange(pos_min - 0.5, pos_max + 1.5, 1)

# ---- Set up the figure ----
fig, ax = plt.subplots(figsize=(7, 4.5))

# Initial histogram for t=0
counts, _, patches = ax.hist(positions_over_time[:, 0], bins=bins, edgecolor="black")
ax.set_title(f"Random Walk — Positions at step 0 (p_right={p_right:.2f})")
ax.set_xlabel("Position")
ax.set_ylabel("Count")

# Keep axis limits stable across frames for a smooth animation
ax.set_xlim(bins[0], bins[-1])
ax.set_ylim(0, max(counts.max(), 1) * 1.1)

# ---- Animation update function ----
def update(frame):
    ax.cla()  # clear the axes for redrawing a single-axes animation
    data = positions_over_time[:, frame]
    counts, _, _ = ax.hist(data, bins=bins, edgecolor="black")
    ax.set_title(f"Random Walk — Positions at step {frame} (p_right={p_right:.2f})")
    ax.set_xlabel("Position")
    ax.set_ylabel("Count")
    ax.set_xlim(bins[0], bins[-1])
    # Dynamic y-limit to fit the distribution at each frame (keeps a single plot, no subplots)
    ax.set_ylim(0, max(counts.max(), 1) * 1.1)
    return []

anim = FuncAnimation(fig, update, frames=positions_over_time.shape[1], interval=80, blit=False, repeat=True)

plt.close(fig)  # prevent duplicate static display

# Display as HTML5 animation (works well in Jupyter)
display(HTML(anim.to_jshtml()))
